In [1]:
# Imports 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from bert.tokenization import FullTokenizer
import tensorflow as tf
from tensorflow.keras import backend as K

max_seq_length = 256
sess = tf.compat.v1.Session()

In [2]:
# Import data
data = pd.read_csv("../../data/mturk_experiment_2.csv",encoding='unicode_escape')
labels = data["Formality"]
samples = data["Sentence"]


train_samples, test_samples, train_labels,test_labels = train_test_split(samples, labels, test_size=0.2)

train_samples = np.array(train_samples)
test_samples = np.array(test_samples)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [3]:
# Utils - from https://github.com/strongio/keras-bert/blob/master/keras-bert.ipynb
class InputExample(object):
    def __init__(self, guid, text_a, text_b=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)

def create_tokenizer_from_hub_module():
    #bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)

    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    tokenizer = FullTokenizer(vocab_file)

    return tokenizer


def convert_text_to_examples(texts, labels):
    """Create InputExamples"""
    InputExamples = []
    for text, label in zip(texts, labels):
        InputExamples.append(
            InputExample(guid=None, text_a=" ".join(text), text_b=None, label=label)
        )
    return InputExamples

In [11]:
# Custom BERT layer
class BertLayer(tf.compat.v1.layers.Layer):
    def __init__(self, n_fine_tune_layers=10, **kwargs):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        '''
        self.bert = hub.Module(
            bert_path,
            trainable=self.trainable,
            name="{}_module".format(self.name)
        )
        trainable_vars = self.bert.variables
        '''
        trainable_vars = bert_layer.variables

        # Remove unused layers
        trainable_vars = [var for var in trainable_vars if not "/cls/" in var.name]
        
        # Select how many layers to fine tune
        trainable_vars = trainable_vars[-self.n_fine_tune_layers :]
        
        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)
        
        # Add non-trainable weights
        for var in bert_layer.variables:
            if var not in bert_layer.trainable_weights:
                bert_layer.non_trainable_weights.append(var)
        

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        
        result = bert_layer(inputs=bert_inputs, signature="tokens", as_dict=True)[
            "pooled_output"
        ]
        
        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [5]:
# TFHub tokenisation
tokeniser = create_tokenizer_from_hub_module()

train_examples = convert_text_to_examples(train_samples, train_labels)
test_examples = convert_text_to_examples(test_samples, test_labels)


In [12]:
# BERT Test
in_id = tf.keras.layers.Input(shape=(max_seq_length,), name="input_ids")
in_mask = tf.keras.layers.Input(shape=(max_seq_length,), name="input_masks")
in_segment = tf.keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
bert_inputs = [in_id, in_mask, in_segment]

# Instantiate the custom Bert Layer defined above
bert_output = BertLayer(n_fine_tune_layers=10)(bert_inputs)

# Build the rest of the classifier 
dense = tf.keras.layers.Dense(256, activation='relu')(bert_output)
pred = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)
model.summary()

TypeError: in user code:

    C:\Users\jack-\AppData\Local\Temp/ipykernel_19956/489344787.py:45 call  *
        result = bert_layer(inputs=bert_inputs, signature="tokens", as_dict=True)[

    TypeError: tf__call() got an unexpected keyword argument 'signature'
